In [19]:
import requests, json
from bs4 import BeautifulSoup

In [20]:
# Liste des adresses IP de l'imprimante
PRINTER_IPS = ['ip-dcp.local','ip-dg.local','ip-dgpecrp.local']
# Liste des noms des cartouches à rechercher
CARTRIDGES = ['Cartouche Jaune', 'Cartouche noir', 'Cartouche Cyan', 'Cartouche Magenta']

In [21]:
def get_printer_status(printer_ip):
    printer_url = f'http://{printer_ip}/info_suppliesStatus.html?tab=Home&menu=SupplyStatus'

    try:
        # Effectuer une requête GET à l'URL
        response = requests.get(printer_url, timeout=0.1)
        response.raise_for_status()  # Vérifier si la requête a réussi
        return response.content
    except requests.exceptions.RequestException as e:
        #print(f"La requête pour l'imprimante {printer_ip} a échoué:", e)
        return None

In [22]:
printer_page = get_printer_status(PRINTER_IPS[1])

In [23]:
def parse_printer_status(content):
    if content is None:
        return None

    try:
        soup = BeautifulSoup(content, 'html.parser')

        cartridges_found = False
        cartridge_status = []

        # Trouver tous les éléments <td> avec la classe CSS "SupplyName width65"
        td_elements = soup.find_all('td', {'class': 'SupplyName width65'})
        for td_element in td_elements:
            nom_cartouche = td_element.contents[0].strip()
            if nom_cartouche in CARTRIDGES:
                # Trouver le prochain élément <td> avec la classe CSS "SupplyName width35 alignRight"
                td_status = td_element.find_next('td', {'class': 'SupplyName width35 alignRight'})
                if td_status:
                    etat_texte = td_status.get_text(strip=True)
                    etat_texte = etat_texte.replace('*', '')

                    # Remove special characters
                    etat_texte = ''.join(c for c in etat_texte if c.isalnum() or c.isspace())

                    # Ajouter l'état de la cartouche à la liste
                    cartridge_status.append({
                        'cartouche': nom_cartouche,
                        'status': etat_texte
                    })

                cartridges_found = True

        if not cartridges_found:
            return []

        return cartridge_status
    except Exception as e:
        print("Une erreur s'est produite lors de l'analyse de la page:", e)
        return None

In [24]:
printer_levels = parse_printer_status(printer_page)
printer_levels

[{'cartouche': 'Cartouche noir',
  'status': '90\n                                        \n                                            '},
 {'cartouche': 'Cartouche Cyan',
  'status': '100\n                            \n                            '},
 {'cartouche': 'Cartouche Magenta',
  'status': '100\n                            \n                            '},
 {'cartouche': 'Cartouche Jaune',
  'status': '100\n                            \n                            '}]

In [25]:
def printer_status(PRINTER_IPS):
    printer_statuses = []

    # Parcourir les adresses IP de l'imprimante
    for printer_ip in PRINTER_IPS:
        # Obtenir l'état de l'imprimante
        response_content = get_printer_status(printer_ip)

        # Analyser l'état de l'imprimante
        parsed_status = parse_printer_status(response_content)

        if parsed_status is not None:
            printer_statuses.append({
                'ip': printer_ip,
                'status': parsed_status
            })

    return printer_statuses

## Sequential code Exce

In [26]:
import re

In [29]:
ps = printer_status(PRINTER_IPS)

In [30]:
# Remove white spaces and special characters from the data
data_json = json.dumps(ps, separators=(',', ':'), ensure_ascii=False)
data_json = re.sub(r'[\s\\n]+', '', data_json)
data_json

'[{"ip":"ip-dcp.local","status":[{"cartouche":"Cartoucheoir","status":"90"},{"cartouche":"CartoucheCya","status":"100"},{"cartouche":"CartoucheMageta","status":"100"},{"cartouche":"CartoucheJaue","status":"100"}]},{"ip":"ip-dg.local","status":[{"cartouche":"Cartoucheoir","status":"100"},{"cartouche":"CartoucheCya","status":"100"},{"cartouche":"CartoucheMageta","status":"100"},{"cartouche":"CartoucheJaue","status":"100"}]},{"ip":"ip-dgpecrp.local","status":[{"cartouche":"Cartoucheoir","status":"90"},{"cartouche":"CartoucheCya","status":"100"},{"cartouche":"CartoucheMageta","status":"100"},{"cartouche":"CartoucheJaue","status":"100"}]}]'